# 自定义层

我们可以茶馆是去自定义一个layer(把模型比作是三明治，那俺么其中的层就是夹的各种肉与菜)

首先定义一个没有任何参数的层(就和一个普通的函数没什么区别)

In [1]:
import torch as tor
import torch.nn.functional as tor_nn_F
from torch import nn as tor_nn

In [2]:
class CenteredLayer(tor_nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [3]:
layer = CenteredLayer()
layer(tor.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

当然，这个也支持套娃和拼接

In [4]:
net = tor_nn.Sequential(tor_nn.Linear(8, 128), CenteredLayer())

Y = net(tor.rand(4, 8))
Y.mean()

tensor(6.5193e-09, grad_fn=<MeanBackward0>)

其实我们就可以看到，在这里，决定权在forward上，也就是说我们的数据如何操作一切均是由forward决定

下面是带参数的网络

In [5]:
class MyLinear(tor_nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = tor_nn.Parameter(tor.randn(in_units, units)) # 直接操作为Paramter，这个其实就是我们的数据的操作
        self.bias = tor_nn.Parameter(tor.randn(units,))            # 直接操作Paramter，其实nn.Sequential实质上是一一样的
    def forward(self, X):
        linear = tor.matmul(X, self.weight.data) + self.bias.data # 其实深度学习的原则也就是在这里
        return tor_nn_F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.1950, -0.4204,  1.1378],
        [-1.2057, -1.0196,  0.8786],
        [ 1.5414, -0.4427, -1.5344],
        [-0.1536, -0.3262,  2.2918],
        [-0.4637,  0.5407, -1.6491]], requires_grad=True)

使用自定义的层进行向前传播计算

In [6]:
linear(tor.rand(2, 5))

tensor([[0.0000, 0.4464, 1.8458],
        [0.0000, 0.6230, 0.0000]])

将自定义的层加入到我们自己的模型中去

In [7]:
net = tor_nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(tor.rand(2, 64))

tensor([[0.],
        [0.]])